In [1]:
# Instalar as bibliotecas necessárias
# !pip install transformers datasets torch

### Importar as bibliotecas necessárias


In [2]:

from datasets import load_dataset
import evaluate
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

2025-11-14 17:12:02.581545: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-14 17:12:02.618009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-14 17:12:03.543534: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Carregar o conjunto de dados IMDB

In [3]:
dataset = load_dataset('imdb')

### Reduzir o conjunto de dados para acelerar o treinamento


In [4]:
small_train_dataset = dataset['train'].select(range(1000))
small_test_dataset = dataset['test'].select(range(1000))

# Carregar o tokenizador e o modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Função para tokenizar os dados

In [5]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenizar o conjunto de dados de treino e teste
small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_test_dataset = small_test_dataset.map(tokenize_function, batched=True)

### Selecionar as colunas de entrada e saída


In [6]:
small_train_dataset = small_train_dataset.rename_column("label", "labels")
small_train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
small_test_dataset = small_test_dataset.rename_column("label", "labels")
small_test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

### Carregar a métrica de acurácia


In [7]:
accuracy_metric = evaluate.load("accuracy")

# Função de avaliação para calcular a acurácia
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

### Configurações do treinamento com otimizações para acelerar o processo


In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Aumentar o tamanho do lote
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Reduzir para 1 época
    weight_decay=0.01,
    fp16=True,  # Habilitar precisão mista
)

### Definir o Trainer


In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics  # Adicionar a função de métricas
)

### Executar o treinamento (fine-tuning)


In [ ]:
trainer.train()

# Avaliar o modelo ajustado
results = trainer.evaluate()

print(f"Fine-tuned Loss: {results['eval_loss']}")
print(f"Fine-tuned Accuracy: {results['eval_accuracy']}")



/home/carloslessa/FCD/POSTECH/modulo4/04-GenerativeIA/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


### Comparar com o modelo original (sem fine-tuning)


In [ ]:
original_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Garantir que o modelo original esteja na GPU (ou na CPU)
original_model.to(training_args.device)

trainer.model = original_model
original_results = trainer.evaluate()

print(f"Original Loss: {original_results['eval_loss']}")
print(f"Original Accuracy: {original_results['eval_accuracy']}")